# hash table概念：
一種結合array與linked-list的資料結構，array裡的每個位置後面都接一條linked-list，而array裡的每個位置都是linked-list的第一個頭節點，將輸入的key(不限型態，但這次的作業規定的是字串)經過MD5來加密轉成10進位的數值，用取餘數的方法(或其他)來將加密過後的key(以下簡稱new_key)放進對應的array儲存空間裡，如果輸入不同的key對應到相同的array位置，就將new_key新增進此位置後面的節點<br>
<br>
參考資料：<br>
https://docs.google.com/presentation/d/e/2PACX-1vT1HO9Nl475k2bR0l1x8_Tr4V5Wzx0BEqp9bpmHckvj8kTeJehhYVlOJUDVPhLQm6kjGCJ_sLMSBUw5/pub?start=false&loop=false&delayms=3000&slide=id.g7565e27c53_0_20<br>
https://github.com/imucici/my-learning-note/blob/master/HW3/binary_search_tree_06170211.py

# hash function說明：
### 加密(encryption):將輸入的key用MD5轉成10進位的數值
我多寫一個把數字加密的fuction，方便我在寫新增、移除、搜尋時可以呼叫此fuction來處理將字串轉成10進位的數值(這裡是參考助教給的程式碼)

![text](https://github.com/angelawu0113/angela/blob/master/week8/encryption.jpg?raw=true)

In [6]:
    def encryption(self,key): #寫一個負責加密fuction:把字串丟進去，用 MD5 來輸出加密數字
        h = MD5.new()
        h.update(key.encode("utf-8"))
        print(h.hexdigest())
        new_key = int(h.hexdigest(),16)
        return new_key #new_key是加密過的數字

### 新增(add):將輸入的key找到對應位置並新增進去，如果該位置已有值，就在後面接一個node(不存重複值)
呼叫加密function，將n存為加密的數字/capacity的餘數(n為放入array的位置)，如果data裡的那個位子是空的，就把 new_key存進去，不然那個位子已有存東西，就看如果那個位置的值不是new_key(這行是為了防止加入重複值)，再來判斷如果 array後面沒有 node，就在他的後面建一個 node，不然array後面有 node，就用while迴圈找到最後一個 node(後來更改迴圈的條件：如果array[n].next有下一個節點，且array[n]不是new_key就一直跑回圈，直到找到最後一個節點)，在判斷最後一個節點是沒有next且最後一個節點不是new_key，就新增new_key在最後一個節點後面

![text](https://github.com/angelawu0113/angela/blob/master/week8/add.jpg?raw=true)

In [7]:
    def add(self, key):
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數放到對應的 array index裡
        if self.data[n] == None: # 如果data裡的那個位子是空的，就把 new_key存進去
            self.data[n] = ListNode(new_key)
        else: #如果那個位子已有存東西
            if self.data[n].val != new_key: #後來加這行來防止加入重複值
                if self.data[n].next == None:#如果 array後面沒有 node，就在他的後面建一個 node
                    self.data[n].next = ListNode(new_key)
                else: # array後面有 node，找到最後一個 node，新增在他後面
                    x = self.data[n] #參考同學的 while走訪概念
                    y = x.next
                    while y.next and y.val != new_key: #後來更改條件來防止重複值
                        y = y.next #現在 y是 linked-list的最後一個 node
                    if y.next == None and y.val != new_key:  
                        y.next = ListNode(new_key)

### 移除(remove):
呼叫加密function，將n存為加密的數字/capacity的餘數(n為放入array的位置)，找到那個值，把前一個node指向下一個node，如果要刪除的值在array裡就判斷他有沒有next，如果有他的下一個就用下一個取代他成為新的頭，如果沒有就變None

![text](https://github.com/angelawu0113/angela/blob/master/week8/remove.jpg?raw=true)

In [8]:
    def remove(self, key):
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數為了放到對應的array index裡
        if self.data[n] != None: #如果array[n]不是空
            if self.data[n].val == new_key: #如果要移除的 new_key就在 array裡
                if self.data[n].next != None: # array後面有node，把 array的下一個 node存入array原來的位置
                    self.data[n] = self.data[n].next
                else: # array後面沒有node，直接存成 None
                    self.data[n] = None 
            elif self.data[n].next and self.data[n].val != new_key: #如果後面有 linked-list且array裡的值不是要找的值,就使用 while找到要移除的 new_key
                x = self.data[n] #參考同學的 while走訪概念
                y = x.next
                while y != None and y.val != new_key: #設這個條件是確保迴圈停止的條件不是走訪完沒找到，不然就是找到
                    x = y
                    y = y.next #現在 y是那個 new_key， x是 y的前一個 node  
                if y: #判斷如果y存在，就把前一個 node指向y的下一個 node
                    x.next = y.next   

### 搜尋(contains):
呼叫加密function，將n存為加密的數字/capacity的餘數(n為放入array的位置)，從頭開始找，找到就回傳True，找不到就往下找直到整條linked-list找完都找不到就回傳False

![text](https://github.com/angelawu0113/angela/blob/master/week8/contains.jpg?raw=true)

In [9]:
    def contains(self, key):
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數來決定 key在array的哪裡
        if self.data[n] == None:#如果 array裡的那個位置沒有值，就回傳False
            return False
        elif self.data[n].val != None: #如果 array裡的那個位置有值
            if self.data[n].val == new_key: # key就在 array，就回傳True
                return True
            else: # key不在 array裡
                if self.data[n].next != None: #如果 array後面有 linked-list,使用 while找到key
                    x = self.data[n] #參考同學的 while走訪概念
                    y = x.next
                    while new_key != y.val and y.next: #用while找到對應的值                               
                        y = y.next #現在 y是那個 key， x是 y的前一個 node  
                    if y.val == new_key:
                        return True
                    else: 
                        return False 
                else: # array後面沒有linked-list
                    return False 

# 學習歷程：
這次我寫的方式就是想到什麼就直接轉成程式碼寫進去試試看，這是我一開始的版本：
一開始我想的沒有那麼周全，我的add也沒有防止重複值，所以我可以add重複值進去，雖然這次作業沒有特別規定不能加重複值進去，但我remove就不知道該怎麼寫可以把全部的重複值刪除，所以我測值輸入了2個重複值，刪除了一個，最後contains出來就是True

In [10]:
from Cryptodome.Hash import MD5
class ListNode:
    def __init__(self, val):
        self.val = val
        self.next = None
        """
        :type val: int
        :type next: ListNode
        :rtype: None        
        """
        
class MyHashSet:
    def __init__(self, capacity = 5):
        self.capacity = capacity
        self.data = [None] * capacity #array
        """
        :type capacity: int
        :rtype: None
        """
    def encryption(self,key): #寫一個負責加密fuction:把字串丟進去，用 MD5 來輸出加密數字
        h = MD5.new()
        h.update(key.encode("utf-8"))
        print(h.hexdigest())
        new_key = int(h.hexdigest(),16)
        return new_key #new_key是加密過的數字
    
    def add(self, key):
        """
        :type key: str
        :rtype: None
        """
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數放到對應的 array index裡
        if self.data[n] == None: # 如果data裡的那個位子是空的，就把 new_key存進去
            self.data[n] = ListNode(new_key)
        else: #如果那個位子已有存東西
            if self.data[n].next == None:#如果 array後面沒有 node，就在他的後面建一個 node
                self.data[n].next = ListNode(new_key)
            else: # array後面有 node，找到最後一個 node，新增在他後面
                x = self.data[n]
                y = x.next
                while y.next:
                    y = y.next #現在 y是 linked-list的最後一個 node
                y.next = ListNode(new_key)         
               
    def remove(self, key):
        """
        :type key: str
        :rtype: None
        """
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數為了放到對應的array index裡
        if self.data[n].val == new_key: #如果要移除的 new_key就在 array裡
            if self.data[n].next != None: # array後面有node，把 array的下一個 node存入array原來的位置
                self.data[n] = self.data[n].next
            else: # array後面沒有node，直接存成 None
                self.data[n].val = None       
        elif self.data[n].next: #如果後面有 linked-list,使用 while找到要移除的 new_key
            x = self.data[n]
            y = x.next
            while y.val != new_key and y.next: 
                x = y
                y = y.next #現在 y是那個 new_key， x是 y的前一個 node  
            if y.next: #如果後面還有 node
                x.next = y.next 
                y = None
                MyHashSet().remove(key) #處理重複值再呼叫一次移除函式處理重複值
            else: #如果要移除的就是linked-list最後一個node
                y = None
        
    def contains(self, key):
        """
        :type key: str
        :rtype: bool(True or False)
        """
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數來決定 key在array的哪裡
        if self.data[n] == None: #如果 array裡的那個位置沒有值
            return False
        elif self.data[n].val != None: #如果 array裡的那個位置有值
            if self.data[n].val == new_key: # key就在 array
                return True
            else: # key不在 array裡
                if self.data[n].next != None: #如果 array後面有 linked-list,使用 while找到key
                    x = self.data[n]
                    y = x.next
                    while new_key != y.val and y.next:                                
                        y = y.next #現在 y是那個 key， x是 y的前一個 node  
                    if y.val == new_key:
                        return True
                    else: 
                        return False 
                else: # array後面沒有linked-list
                    return False

In [11]:
hashSet = MyHashSet()
hashSet.add("nt")
hashSet.add("med")
hashSet.add("nt")
hashSet.add("med")
hashSet.remove("med")

print(hashSet.contains("nt"),
hashSet.contains("med"),
hashSet.contains("angela"),hashSet.contains("bird"))

25930e3036f13852cb0b29694bbab611
de4667d1f44423b565b07a7bb14790fc
25930e3036f13852cb0b29694bbab611
de4667d1f44423b565b07a7bb14790fc
de4667d1f44423b565b07a7bb14790fc
25930e3036f13852cb0b29694bbab611
de4667d1f44423b565b07a7bb14790fc
36388794be2cf5f298978498ff3c64a2
abaecf8ca3f98dc13eeecbac263cd3ed
True True False False


後來我想說那我就在add防止重複值，那我remove就可以不用花時間去找有沒有重複值，這樣比較容易成功，所以我就開始重寫了add跟remove，於是有了第二個版本:最後我測出來的如果加了重複值，移除後contains最後跑出False(成功)，加值進去contains出True(成功)，contains沒加進去的數跑出False(成功)

In [12]:
from Cryptodome.Hash import MD5
class ListNode:
    def __init__(self, val):
        self.val = val
        self.next = None
        
        
class MyHashSet:
    def __init__(self, capacity = 5):
        self.capacity = capacity
        self.data = [None] * capacity #array
       
        
    def encryption(self,key): #寫一個負責加密fuction:把字串丟進去，用 MD5 來輸出加密數字，這是參考老師投影片助教給的程式碼
        h = MD5.new()
        h.update(key.encode("utf-8"))
        new_key = int(h.hexdigest(),16)
        return new_key #new_key是加密過的數字
    
    def add(self, key):
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數放到對應的 array index裡
        if self.data[n] == None: # 如果data裡的那個位子是空的，就把 new_key存進去
            self.data[n] = ListNode(new_key)
        else: #如果那個位子已有存東西
            if self.data[n].val != new_key: #後來加這行來防止加入重複值
                if self.data[n].next == None:#如果 array後面沒有 node，就在他的後面建一個 node
                    self.data[n].next = ListNode(new_key)
                else: # array後面有 node，找到最後一個 node，新增在他後面
                    x = self.data[n] #參考同學的 while走訪概念
                    y = x.next
                    while y.next and y.val != new_key: #後來更改條件來防止重複值
                        y = y.next #現在 y是 linked-list的最後一個 node
                    if y.next == None and y.val != new_key:  
                        y.next = ListNode(new_key)
                
    def remove(self, key):
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數為了放到對應的array index裡
        if self.data[n] != None:
            if self.data[n].val == new_key: #如果要移除的 new_key就在 array裡
                if self.data[n].next != None: # array後面有node，把 array的下一個 node存入array原來的位置
                    self.data[n] = self.data[n].next
                else: # array後面沒有node，直接存成 None
                    self.data[n] = None 
            elif self.data[n].next and self.data[n].val != new_key: #如果後面有 linked-list且array裡的值不適要找的值,就使用 while找到要移除的 new_key
                x = self.data[n] #參考同學的 while走訪概念
                y = x.next
                while y != None and y.val != new_key: 
                    x = y
                    y = y.next #現在 y是那個 new_key， x是 y的前一個 node  
                if y: 
                    x.next = y.next        
        
    def contains(self, key):
        new_key = MyHashSet().encryption(key) #呼叫 encryption()來把 key 加密
        n = new_key % self.capacity #取餘數來決定 key在array的哪裡
        if self.data[n] == None:#如果 array裡的那個位置沒有值
            return False
        elif self.data[n].val != None: #如果 array裡的那個位置有值
            if self.data[n].val == new_key: # key就在 array
                return True
            else: # key不在 array裡
                if self.data[n].next != None: #如果 array後面有 linked-list,使用 while找到key
                    x = self.data[n] #參考同學的 while走訪概念
                    y = x.next
                    while new_key != y.val and y.next:                                
                        y = y.next #現在 y是那個 key， x是 y的前一個 node  
                    if y.val == new_key:
                        return True
                    else: 
                        return False 
                else: # array後面沒有linked-list
                    return False 

In [13]:
hashSet = MyHashSet()
hashSet.add("med")
hashSet.add("med")
hashSet.add("d")
hashSet.add("c")

hashSet.remove("med")
hashSet.remove("d")

print(
hashSet.contains("c"),
hashSet.contains("d"),
hashSet.contains("med"),
hashSet.contains("a")
)

True False False False


# 心得：
經過這幾次作業後，這次寫就覺得比較簡單，也越來越能把自己的邏輯轉換成程式碼了